#Первая версия кода для контеста

In [113]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb

from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [114]:
features = pandas.read_csv('data/features.csv', index_col='match_id')
features_test = pandas.read_csv('data/features_test.csv', index_col='match_id')

In [115]:
matches = features.copy().fillna(0)
test_matches = features_test.copy().fillna(0)
# matches = features.copy().fillna(0)
# train_size = int(matches.shape[0] * 2 / 3)
# test_matches = matches[train_size:]
# matches = matches[:train_size]
# y_test = test_matches['radiant_win']
# del test_matches['duration']
# del test_matches['radiant_win']
# del test_matches['tower_status_dire']
# del test_matches['tower_status_radiant']
# del test_matches['barracks_status_radiant']
# del test_matches['barracks_status_dire']

In [116]:
fields = ['level', 'gold', 'xp', 'lh', 'kills', 'deaths', 'items']
teams = ['r', 'd']
players = range(1, 6)
for field in fields:
    for team in teams:
        name = team + '_' + field
        matches[name] = 0
        test_matches[name] = 0
        for player in players:
            matches[name] += matches[team + str(player) + '_' + field] 
            test_matches[name] += test_matches[team + str(player) + '_' + field] 
            del matches[team + str(player) + '_' + field] 
            del test_matches[team + str(player) + '_' + field] 

In [117]:
def preprocess(data, cat_cols, other_cols, train_size=None):
    if train_size is None:
        train_size = int(2 * data.shape[0] / 3)
    if train_size == 0:
        transformer = DictVectorizer(sparse=False)
        data_arr = transformer.fit_transform(data[cat_cols].astype(str).T.to_dict().values())
        train = np.hstack((data_arr, data[other_cols]))
#         train = data
#         print(list(data.columns))
        if 'radiant_win' in list(data.columns):
            y_train = np.array(data['radiant_win'])
            del data['duration']
            del data['radiant_win']
            del data['tower_status_dire']
            del data['tower_status_radiant']
            del data['barracks_status_radiant']
            del data['barracks_status_dire']
            return train, y_train
        return train
    
    y_train = np.array(data['radiant_win'])
    y_test = y_train[train_size:]
    y_train = y_train[:train_size]
    del data['duration']
    del data['radiant_win']
    del data['tower_status_dire']
    del data['tower_status_radiant']
    del data['barracks_status_radiant']
    del data['barracks_status_dire']
    transformer = DictVectorizer(sparse=False)
    data_arr = transformer.fit_transform(data[cat_cols].astype(str).T.to_dict().values())
    data = np.hstack((data_arr, data[other_cols]))
    test = data[train_size:, :]
    train = data[:train_size, :]
    return train, y_train, test, y_test

##Логистическая регрессия на выбранных героях

In [118]:
N = 113
data = matches
X_pick = np.zeros((data.shape[0], N))
for i, match_id in enumerate(data.index):
    for p in range(5):
        X_pick[i, data.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, data.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1
for i, match_id in enumerate(data.index):
    for p in range(5):
        X_pick[i, data.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, data.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

In [119]:
train = X_pick
y_train = matches['radiant_win']

In [120]:
N = 113
data = test_matches
X_pick = np.zeros((data.shape[0], N))
for i, match_id in enumerate(data.index):
    for p in range(5):
        X_pick[i, data.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, data.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1
for i, match_id in enumerate(data.index):
    for p in range(5):
        X_pick[i, data.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, data.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

In [121]:
test = X_pick

In [122]:
logreg = LogisticRegression()
logreg.fit(train, y_train)
logreg_train_preds = logreg.predict_proba(train)[:, 1]
logreg_test_preds = logreg.predict_proba(test)[:, 1]

##xgboost

In [123]:
matches['log_heroes'] = logreg_train_preds
test_matches['log_heroes'] = logreg_test_preds

In [124]:
bad_columns = ['start_time', 'duration', 'radiant_win', 'tower_status_dire', 'tower_status_radiant',
               'barracks_status_radiant', 'barracks_status_dire', 'first_blood_player1', 'first_blood_player2']
categorical_columns = ['lobby_type']
for i in range(1, 6):
    bad_columns.append('r'+str(i)+'_hero')
    bad_columns.append('d'+str(i)+'_hero')
other_cols = [col for col in matches.columns if not (col in categorical_columns)
             and not (col in bad_columns)]

In [125]:
print(list(matches.columns))

['start_time', 'lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero', 'first_blood_time', 'first_blood_team', 'first_blood_player1', 'first_blood_player2', 'radiant_bottle_time', 'radiant_courier_time', 'radiant_flying_courier_time', 'radiant_tpscroll_count', 'radiant_boots_count', 'radiant_ward_observer_count', 'radiant_ward_sentry_count', 'radiant_first_ward_time', 'dire_bottle_time', 'dire_courier_time', 'dire_flying_courier_time', 'dire_tpscroll_count', 'dire_boots_count', 'dire_ward_observer_count', 'dire_ward_sentry_count', 'dire_first_ward_time', 'duration', 'radiant_win', 'tower_status_radiant', 'tower_status_dire', 'barracks_status_radiant', 'barracks_status_dire', 'r_level', 'd_level', 'r_gold', 'd_gold', 'r_xp', 'd_xp', 'r_lh', 'd_lh', 'r_kills', 'd_kills', 'r_deaths', 'd_deaths', 'r_items', 'd_items', 'log_heroes']


In [126]:
train, y_train = preprocess(matches, categorical_columns, other_cols, train_size=0)
test = preprocess(test_matches, categorical_columns, other_cols, train_size=0)

In [127]:
train.shape

(97230, 36)

In [128]:
test.shape

(17177, 36)

In [129]:
dtrain = xgb.DMatrix(train, label=y_train)
dtest = xgb.DMatrix(test)

In [130]:
param = {'bst:max_depth':3, 'bst:eta':1, 'silent':1, 'objective':'binary:logistic' }
param['nthread'] = 4
param['eval_metric'] = 'auc'
evallist  = [(dtrain,'train')]

In [131]:
num_round = 10
bst = xgb.train(param, dtrain, num_round)

In [132]:
boost_preds = bst.predict(dtest,ntree_limit=bst.best_ntree_limit)

##Random Forest

In [133]:
forest = RandomForestClassifier(n_estimators=100)
forest.fit(train, y_train)
forest_preds = forest.predict_proba(test)[:, 1]


In [134]:
preds = 0.5 * (boost_preds + forest_preds)

In [135]:
test_matches['radiant_win'] = preds

In [138]:
final = test_matches[['radiant_win']]

In [140]:
final.to_csv('answer.csv')